In [ ]:
import csv
import json
import io
import sys

# --- Phần 1: Tăng giới hạn kích thước trường CSV ---
max_int = sys.maxsize
print("Đang tìm giới hạn kích thước trường CSV tối đa mà hệ thống hỗ trợ...")
while True:
    try:
        csv.field_size_limit(max_int)
        print(f"Đã đặt giới hạn kích thước trường CSV thành: {max_int}")
        break
    except OverflowError:
        max_int = int(max_int / 10)

# --- Phần 2: Đặt tên file và xử lý ---
input_filename = './src/conference/examples/determine_links_train_data.csv' # Tên file CSV đầu vào của bạn
# Đổi tên file output một lần nữa
output_filename = 'output_transformed_final.csv'

print(f"Đang xử lý file: {input_filename}")

try:
    with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
         open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)

        try:
            header = next(csv_reader)
            csv_writer.writerow(header)
            print(f"Đã đọc header: {header}")

            for i, row in enumerate(csv_reader):
                current_line_num = i + 2
                if len(row) == 2:
                    input_text = row[0]
                    output_json_str = row[1]
                    transformed_output = ""

                    try:
                        data_dict = json.loads(output_json_str)

                        if isinstance(data_dict, dict):
                            # ---- THAY ĐỔI Ở ĐÂY ----
                            lines = [] # Danh sách để lưu các dòng đã định dạng
                            num_items = len(data_dict) # Lấy số lượng cặp key-value

                            # Sử dụng enumerate để có chỉ số (idx)
                            for idx, (key, value) in enumerate(data_dict.items()):
                                # Tạo dòng cơ bản
                                line = f'"{key}": "{value}"'
                                # Nếu không phải là mục cuối cùng, thêm dấu phẩy
                                if idx < num_items - 1:
                                    line += ","
                                lines.append(line) # Thêm dòng vào danh sách

                            # Nối các dòng trong danh sách bằng ký tự xuống dòng
                            transformed_output = "\n".join(lines)
                            # ---- KẾT THÚC THAY ĐỔI ----
                        else:
                            print(f"Cảnh báo dòng {current_line_num}: Cột output không phải là JSON object: {output_json_str[:100]}...")
                            transformed_output = output_json_str

                    except json.JSONDecodeError:
                        print(f"Cảnh báo dòng {current_line_num}: Không thể parse JSON từ cột output: {output_json_str[:100]}...")
                        transformed_output = output_json_str
                    except Exception as e:
                        print(f"Lỗi xử lý JSON ở dòng {current_line_num}: {e}")
                        transformed_output = output_json_str

                    csv_writer.writerow([input_text, transformed_output])

                else:
                    print(f"Cảnh báo dòng {current_line_num}: Dòng không có đúng 2 cột. Bỏ qua: {row}")

        except csv.Error as e:
            print(f"Lỗi CSV ở dòng {csv_reader.line_num}: {e}")
            print("Kiểm tra lại cấu trúc file CSV hoặc tăng 'field_size_limit' nếu cần.")
        except StopIteration:
             print("File CSV không có dữ liệu sau header.")
        except Exception as e:
             line_num_info = f" (possibly near line {csv_reader.line_num})" if 'csv_reader' in locals() and hasattr(csv_reader, 'line_num') else ""
             print(f"Đã xảy ra lỗi không mong muốn trong quá trình xử lý file{line_num_info}: {e}")

    print(f"\nXử lý hoàn tất. Dữ liệu đã được ghi vào file: {output_filename}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_filename}'")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn bên ngoài quá trình đọc CSV: {e}")

In [ ]:
import csv
import re
import sys

# --- Tăng giới hạn CSV nếu cần ---
max_int = sys.maxsize
print("Đang tìm giới hạn kích thước trường CSV tối đa...")
while True:
    try:
        csv.field_size_limit(max_int)
        print(f"Đã đặt giới hạn kích thước trường CSV thành: {max_int}")
        break
    except OverflowError:
        max_int = int(max_int / 10)

# --- Đặt tên file ---
input_filename = 'output_transformed_final.csv' # File input từ bước trước
# File output mới, tên rõ ràng hơn
output_filename = 'final_truncated_input_at_4.csv'

# --- Biểu thức chính quy để tìm điểm cắt (CHỈ SỐ 4) ---
# Thay đổi ở đây: thay \d+ bằng số 4 cụ thể
pattern_to_find = re.compile(r"\s*4\.\s+Website of ") # Tìm "4. Website of "

print(f"Đang đọc file: {input_filename}")
print(f"Sẽ ghi kết quả vào: {output_filename}")

try:
    with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
         open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)

        try:
            # Đọc và ghi header
            header = next(csv_reader)
            csv_writer.writerow(header)
            print(f"Đã đọc header: {header}")

            # Xử lý từng dòng dữ liệu CSV
            for i, row in enumerate(csv_reader):
                current_csv_line_num = i + 2 # Số dòng trong file CSV

                if len(row) == 2:
                    input_text_original = row[0]
                    output_text = row[1] # Giữ nguyên cột output

                    # Tìm vị trí đầu tiên của pattern "4. Website of "
                    match = pattern_to_find.search(input_text_original)

                    if match:
                        # Nếu tìm thấy, lấy vị trí bắt đầu
                        start_index = match.start()
                        # Cắt chuỗi từ đầu đến vị trí đó
                        input_text_cleaned = input_text_original[:start_index].rstrip()
                        # Thêm dấu xuống hàng ở cuối sau khi cắt
                        input_text_cleaned += '\n'
                        # print(f"  (CSV Line {current_csv_line_num}) Đã cắt bỏ từ '4. Website of ' tại vị trí {start_index}.")
                    else:
                        # Nếu không tìm thấy "4. Website of ", giữ nguyên và thêm dấu xuống hàng
                        input_text_cleaned = input_text_original + '\n'


                    # Ghi dòng đã xử lý
                    csv_writer.writerow([input_text_cleaned, output_text])

                else:
                    print(f"Cảnh báo dòng CSV {current_csv_line_num}: Dòng không có đúng 2 cột. Bỏ qua: {row}")

        except csv.Error as e:
            print(f"Lỗi CSV ở dòng {csv_reader.line_num}: {e}")
        except StopIteration:
             print("File CSV không có dữ liệu sau header.")
        except Exception as e:
             line_num_info = f" (possibly near line {csv_reader.line_num})" if 'csv_reader' in locals() and hasattr(csv_reader, 'line_num') else ""
             print(f"Đã xảy ra lỗi không mong muốn trong quá trình xử lý file{line_num_info}: {e}")

    print(f"\nXử lý hoàn tất. Dữ liệu đã được cắt (chỉ tại '4. Website of ') và ghi vào file: {output_filename}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_filename}'. Hãy đảm bảo file input có tên này.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn bên ngoài quá trình đọc CSV: {e}")

In [3]:
import csv

# Định nghĩa các giới hạn độ dài để dễ thay đổi
GIOI_HAN_INPUT = 40000
GIOI_HAN_OUTPUT = 5000
DO_DAI_CHUOI_NGAN = 80 # Số ký tự hiển thị khi báo lỗi

def kiem_tra_do_dai_csv(duong_dan_file_csv):
    """
    Đọc file CSV, in độ dài của mỗi cột trong mỗi dòng.
    Đối với cột 'input:' và 'output:', kiểm tra và thông báo ngay nếu vượt quá giới hạn.

    Args:
        duong_dan_file_csv (str): Đường dẫn đến file CSV.

    Returns:
        list: Một danh sách các thông báo lỗi vượt quá độ dài (dùng cho tóm tắt cuối cùng).
              Mỗi thông báo lỗi là một tuple (so_dong, cot, do_dai_thuc_te, chuoi_ngan).
              Trả về danh sách rỗng nếu không có lỗi vượt quá độ dài.
              Các thông báo lỗi khác (ví dụ: lỗi đọc file) cũng có thể được trả về.
    """

    cac_loi_vuot_qua_do_dai = []
    print(f"--- Bắt đầu kiểm tra file: {duong_dan_file_csv} ---")

    try:
        with open(duong_dan_file_csv, 'r', encoding='utf-8') as file_csv:
            doc_csv = csv.DictReader(file_csv)
            ten_cac_cot = doc_csv.fieldnames
            if not ten_cac_cot:
                 print("Lỗi: File CSV trống hoặc không có header.")
                 return [("File", "Không có header", duong_dan_file_csv, "")]

            print(f"Các cột trong file: {', '.join(ten_cac_cot)}")
            print(f"Kiểm tra độ dài: input: > {GIOI_HAN_INPUT}, output: > {GIOI_HAN_OUTPUT}")
            print("-" * 30)

            for so_dong, dong in enumerate(doc_csv, start=2):  # Bắt đầu từ dòng 2
                print(f"--- Dòng {so_dong} ---")
                try:
                    for ten_cot in ten_cac_cot:
                        gia_tri_cot = dong.get(ten_cot)
                        do_dai_cot = len(gia_tri_cot) if gia_tri_cot is not None else 0

                        # In tên cột và độ dài cơ bản
                        print(f"  Cột '{ten_cot}': Độ dài = {do_dai_cot}", end='') # Dùng end='' để không xuống dòng ngay

                        # Kiểm tra và thông báo ngay nếu là cột 'input:' hoặc 'output:' và vượt giới hạn
                        vi_pham = False
                        if ten_cot == 'input:' and gia_tri_cot is not None and do_dai_cot > GIOI_HAN_INPUT:
                            print(f" *** VƯỢT QUÁ GIỚI HẠN ({GIOI_HAN_INPUT}) ***", end='')
                            vi_pham = True
                            chuoi_ngan = gia_tri_cot[:DO_DAI_CHUOI_NGAN]
                            cac_loi_vuot_qua_do_dai.append((so_dong, ten_cot, do_dai_cot, chuoi_ngan))

                        elif ten_cot == 'output:' and gia_tri_cot is not None and do_dai_cot > GIOI_HAN_OUTPUT:
                            print(f" *** VƯỢT QUÁ GIỚI HẠN ({GIOI_HAN_OUTPUT}) ***", end='')
                            vi_pham = True
                            chuoi_ngan = gia_tri_cot[:DO_DAI_CHUOI_NGAN]
                            cac_loi_vuot_qua_do_dai.append((so_dong, ten_cot, do_dai_cot, chuoi_ngan))

                        # Xuống dòng sau khi đã in thông tin cột và thông báo (nếu có)
                        print()

                except Exception as e:
                    # In lỗi xử lý dòng và tiếp tục dòng tiếp theo
                    print(f"\n  Lỗi không xác định khi xử lý các cột trong dòng {so_dong}: {e}")
                    cac_loi_vuot_qua_do_dai.append((so_dong, 'Lỗi xử lý dòng', str(e), ""))
                    # Vẫn cần in dấu phân cách cho dòng tiếp theo nếu lỗi xảy ra giữa chừng
                    if so_dong > 1: print("-" * 10) # Phân cách nhỏ hơn nếu lỗi

            print("-" * 30) # Dòng phân cách cuối cùng của phần duyệt dòng

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv}'.")
        return [("File", "Không tìm thấy file", duong_dan_file_csv, "")]

    except Exception as e:
        print(f"Lỗi không xác định khi đọc file CSV: {e}")
        return [("File", "Lỗi đọc file", str(e), "")]

    print("--- Kết thúc kiểm tra chi tiết từng dòng ---")
    return cac_loi_vuot_qua_do_dai


# --- Phần ví dụ sử dụng và in tóm tắt lỗi (giữ nguyên hoặc điều chỉnh nếu cần) ---
duong_dan_file = "./src/conference/examples/extract_infor_edit_100.csv" # Thay đổi đường dẫn file của bạn ở đây
cac_loi = kiem_tra_do_dai_csv(duong_dan_file)

# In tóm tắt các lỗi VƯỢT QUÁ ĐỘ DÀI đã phát hiện
if cac_loi:
    # Lọc ra các lỗi không phải là lỗi xử lý dòng hoặc lỗi file
    loi_do_dai_thuc_su = [loi for loi in cac_loi if loi[1] not in ('Lỗi xử lý dòng', 'Không tìm thấy file', 'Lỗi đọc file', 'Không có header')]

    if loi_do_dai_thuc_su:
        print(f"\n--- Tóm tắt các dòng vượt quá độ dài cho phép ({len(loi_do_dai_thuc_su)} lỗi) ---")
        for so_dong, cot, do_dai, chuoi_ngan in loi_do_dai_thuc_su:
             # Đảm bảo chuoi_ngan không quá dài khi in
             chuoi_ngan_hien_thi = (chuoi_ngan[:DO_DAI_CHUOI_NGAN] + '...') if len(chuoi_ngan) > DO_DAI_CHUOI_NGAN else chuoi_ngan
             print(f"  - Dòng {so_dong}, cột '{cot}': Độ dài {do_dai}, {DO_DAI_CHUOI_NGAN} ký tự đầu: '{chuoi_ngan_hien_thi}'")
    else:
         # Kiểm tra xem có lỗi nào khác không (ví dụ: lỗi đọc file, xử lý dòng)
         loi_khac = [loi for loi in cac_loi if loi not in loi_do_dai_thuc_su]
         if not loi_khac:
              print("\nKhông có dòng nào vượt quá độ dài cho phép và không có lỗi xử lý nào khác.")
         else:
              print("\nKhông có dòng nào vượt quá độ dài cho phép, nhưng có các lỗi khác:")
              for so_dong, cot, chi_tiet, _ in loi_khac:
                   # Xử lý trường hợp lỗi file không có số dòng
                   dong_str = f"(Dòng {so_dong})" if isinstance(so_dong, int) else f"({so_dong})"
                   print(f"  - {cot} {dong_str}: {chi_tiet}")

else:
    # Trường hợp hàm trả về list rỗng (không có lỗi gì cả)
    print("\nKhông có dòng nào vượt quá độ dài cho phép và không có lỗi nào xảy ra trong quá trình xử lý.")

--- Bắt đầu kiểm tra file: ./src/conference/examples/extract_infor_edit_100.csv ---
Các cột trong file: input:, output:
Kiểm tra độ dài: input: > 40000, output: > 5000
------------------------------
--- Dòng 2 ---
  Cột 'input:': Độ dài = 6197
  Cột 'output:': Độ dài = 595
--- Dòng 3 ---
  Cột 'input:': Độ dài = 776
  Cột 'output:': Độ dài = 28
--- Dòng 4 ---
  Cột 'input:': Độ dài = 12569
  Cột 'output:': Độ dài = 859
--- Dòng 5 ---
  Cột 'input:': Độ dài = 32765
  Cột 'output:': Độ dài = 624
--- Dòng 6 ---
  Cột 'input:': Độ dài = 10023
  Cột 'output:': Độ dài = 942
--- Dòng 7 ---
  Cột 'input:': Độ dài = 13624
  Cột 'output:': Độ dài = 1805
--- Dòng 8 ---
  Cột 'input:': Độ dài = 14582
  Cột 'output:': Độ dài = 1696
--- Dòng 9 ---
  Cột 'input:': Độ dài = 17937
  Cột 'output:': Độ dài = 1165
--- Dòng 10 ---
  Cột 'input:': Độ dài = 3551
  Cột 'output:': Độ dài = 231
--- Dòng 11 ---
  Cột 'input:': Độ dài = 7699
  Cột 'output:': Độ dài = 1035
--- Dòng 12 ---
  Cột 'input:': Độ dài = 

In [ ]:
import csv
import json

def kiem_tra_csv_va_json(duong_dan_file_csv, context_size=50):
    """
    Đọc file CSV, kiểm tra xem cột "output:" có phải là JSON hợp lệ hay không,
    và log dòng lỗi ra cùng với context xung quanh vị trí lỗi.

    Args:
        duong_dan_file_csv (str): Đường dẫn đến file CSV.
        context_size (int): Số lượng ký tự trước và sau vị trí lỗi để hiển thị.

    Returns:
        list: Một danh sách các thông báo lỗi. Mỗi thông báo lỗi là một tuple
              (so_dong, cot, thong_tin_loi). Trả về danh sách rỗng nếu không có lỗi.
    """

    cac_loi = []

    try:
        with open(duong_dan_file_csv, 'r', encoding='utf-8') as file_csv:
            doc_csv = csv.DictReader(file_csv)
            for so_dong, dong in enumerate(doc_csv, start=2):  # Bắt đầu từ dòng 2 (bỏ qua header)
                try:
                    input_text = dong.get('input:')
                    output_text = dong.get('output:')

                    # Kiểm tra JSON
                    if output_text is not None:
                        try:
                            json.loads(output_text)
                        except json.JSONDecodeError as e:
                            # Log dòng lỗi ra và context
                            position = e.pos
                            start = max(0, position - context_size)
                            end = min(len(output_text), position + context_size)
                            context = output_text[start:end]

                            print(f"Lỗi JSON ở dòng {so_dong}: {e}")
                            print(f"Vị trí lỗi: {position}")
                            print(f"Context: ...{context}...")
                            cac_loi.append((so_dong, 'output:', f"Lỗi JSON: {e}"))


                except Exception as e:
                    print(f"Lỗi khi xử lý dòng {so_dong}: {e}")
                    cac_loi.append((so_dong, 'Lỗi xử lý dòng', str(e)))  # Thêm thông tin lỗi vào danh sách

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại '{duong_dan_file_csv}'.")
        return [("File", "Không tìm thấy file", duong_dan_file_csv)]  # Trả về một list với thông báo lỗi

    except Exception as e:
        print(f"Lỗi không xác định khi đọc file CSV: {e}")
        return [("File", "Lỗi đọc file", str(e))]

    return cac_loi


# Ví dụ sử dụng
duong_dan_file = './fine_tune_extract_infor.csv'  # Thay đổi đường dẫn file của bạn ở đây
cac_loi = kiem_tra_csv_va_json(duong_dan_file)

if cac_loi:
    print("Các lỗi được tìm thấy:")
    for so_dong, cot, thong_tin_loi in cac_loi:
        print(f"  - Dòng {so_dong}, cột '{cot}': {thong_tin_loi}")
else:
    print("Không có lỗi nào được tìm thấy.")